# 🐉 Chapter 4 练习 3: DragonNet - 因果推断的「三头龙」

## 从 TARNet 到 DragonNet: 一个巧妙的升级

上一节我们学习了 TARNet，它就像一个双语翻译官。但如果这个翻译官同时还能**判断客户来自哪个国家**（倾向得分），翻译效果会不会更好呢？

这就是 **DragonNet** 的核心思想！

---

## 🎯 学习目标

1. 理解倾向得分头的作用
2. 实现 DragonNet 架构（三头龙！）
3. 掌握 DragonNet 的复合损失函数
4. 理解 Targeted Regularization 的魔法

## 🌟 DragonNet 的直觉: 三头龙的故事

### 问题回顾

在因果推断中，**混淆**是最大的敌人。想象你在研究「广告是否增加用户购买」：

- 经常购物的用户**更容易看到广告**（算法推荐）
- 经常购物的用户**本来就会买更多**

这就是**选择偏差**：处理分配不是随机的，而是依赖于混淆因子。

### TARNet 的局限

TARNet 只学习「预测结果」，但不显式地理解「谁会被处理」。这就像一个翻译官虽然能翻译，但不了解客户的背景。

### DragonNet 的解决方案

DragonNet 添加了**第三个头**——倾向得分头！

```
                       🧠 共享表示层
                           |                    
       +------------------+-------------------+
       |                  |                   |
    🎯 Y(0)头         🎯 Y(1)头          📊 倾向得分头
       |                  |                   |
  控制组结果预测    处理组结果预测      "谁会被处理?"
```

为什么叫「龙网」（DragonNet）？因为这三个头就像传说中的**三头龙**！

### 关键洞察

添加倾向得分头有什么好处？

1. **正则化作用**: 强迫表示层同时学会「预测结果」和「预测处理」
2. **平衡表示**: 让表示层捕获与**处理分配相关的信息**，这正是混淆因子！
3. **Targeted Regularization**: 用倾向得分来调整损失函数，减少偏差

## 📐 核心公式

### DragonNet 架构

$$\Phi(X) = f_{\text{repr}}(X) \quad \text{(共享表示)}$$

$$\hat{Y}(0) = h_0(\Phi(X)), \quad \hat{Y}(1) = h_1(\Phi(X)) \quad \text{(结果头)}$$

$$\hat{e}(X) = h_e(\Phi(X)) \quad \text{(倾向得分头)}$$

### 复合损失函数

$$\mathcal{L}_{\text{DragonNet}} = \mathcal{L}_{\text{factual}} + \alpha \cdot \mathcal{L}_{\text{propensity}} + \beta \cdot \mathcal{L}_{\text{targeted}}$$

其中：

1. **Factual Loss** (和 TARNet 一样):
$$\mathcal{L}_{\text{factual}} = \frac{1}{N}\sum_{i=1}^{N} (Y_i - [T_i \cdot \hat{Y}_i(1) + (1-T_i) \cdot \hat{Y}_i(0)])^2$$

2. **Propensity Loss** (二分类交叉熵):
$$\mathcal{L}_{\text{propensity}} = -\frac{1}{N}\sum_{i=1}^{N} [T_i \log \hat{e}_i + (1-T_i) \log (1-\hat{e}_i)]$$

3. **Targeted Regularization** (DragonNet 的创新):
$$\mathcal{L}_{\text{targeted}} = \frac{1}{N}\sum_{i=1}^{N} \left(Y_i - \hat{Y}_i - \epsilon \cdot h_i\right)^2$$

其中 $h_i = \frac{T_i}{\hat{e}(X_i)} - \frac{1-T_i}{1-\hat{e}(X_i)}$

## 🔮 Targeted Regularization: 魔法公式的直觉

### 那个神秘的 h 是什么？

$$h = \frac{T}{e(X)} - \frac{1-T}{1-e(X)}$$

这个公式来自于**半参数效率理论**和 **TMLE（Targeted Maximum Likelihood Estimation）**。

### 一个直觉解释

想象你在做民意调查，但有些人群更难接触到（比如年轻人不爱接电话）：

- **e(X)** = 某人被调查到的概率（倾向得分）
- **T/e(X)**: 如果被调查到（T=1），用 1/e(X) 加权，相当于「少见的人更重要」
- **-(1-T)/(1-e(X))**: 如果没被调查到（T=0），用反向权重

这个 **h** 本质上是一个**双重鲁棒性**的调整项！

### epsilon 的作用

**epsilon (ε)** 是一个**可学习的标量参数**：

- 它让模型自动学习「需要多少调整」
- 如果模型已经很准，epsilon 会趋近于 0
- 如果有偏差，epsilon 会学到一个非零值来修正

In [ ]:
# 环境设置
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from typing import Tuple, Dict

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 绘图设置
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

print("环境配置完成! 🐉")

## 练习 3.1: 生成强混淆数据

首先，让我们生成一个**强混淆**的数据集。DragonNet 在这种场景下特别有用！

In [ ]:
def generate_confounded_data(
    n: int = 1000,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    生成有强混淆的数据
    
    场景: 广告投放效果评估
    - X: 用户特征（购物频率、平台活跃度、消费能力等）
    - T: 是否看到广告（不是随机分配！高价值用户更容易看到）
    - Y: 是否购买
    
    强混淆: 用户特征同时影响「是否看广告」和「是否购买」
    """
    np.random.seed(seed)
    
    # 用户特征
    X = np.random.randn(n, 5)
    # X[:, 0]: 购物频率
    # X[:, 1]: 平台活跃度
    # X[:, 2]: 消费能力
    # X[:, 3]: 浏览历史
    # X[:, 4]: 点击率
    
    # 强混淆的处理分配（高价值用户更容易看到广告）
    # 真实倾向得分
    propensity = 1 / (1 + np.exp(-(
        1.0 * X[:, 0] +   # 购物频率高 -> 更可能看广告
        0.8 * X[:, 1] +   # 活跃度高 -> 更可能看广告
        0.6 * X[:, 2]     # 消费能力高 -> 更可能看广告
    )))
    
    # TODO: 根据倾向得分生成处理分配 T
    # 提示: 使用 np.random.binomial(1, propensity)
    T = None  # 你的代码
    
    # TODO: 生成潜在结果
    # Y(0): 不看广告的购买概率/金额
    # Y(1): 看广告的购买概率/金额
    # 提示: 结果也应该依赖于 X（这是混淆！）
    #       Y0 = 2.0 + X[:, 0] + 0.5*X[:, 1] + noise
    #       Y1 = Y0 + 处理效应（可以是异质的）
    
    Y0 = None  # 你的代码
    Y1 = None  # 你的代码
    
    # 观测结果
    Y = np.where(T == 1, Y1, Y0)
    
    return X, T, Y, Y0, Y1

In [ ]:
# 测试数据生成
X, T, Y, Y0, Y1 = generate_confounded_data(n=2000)

if T is not None:
    print("数据生成成功! 🎉")
    print(f"数据形状: X={X.shape}")
    print(f"处理组比例: {T.mean():.2%}")
    print(f"\n真实因果效应:")
    print(f"  真实 ATE: {np.mean(Y1 - Y0):.4f}")
    print(f"  真实 ATT: {np.mean((Y1 - Y0)[T == 1]):.4f}")
    print(f"\n朴素估计（有偏！）:")
    naive_ate = Y[T == 1].mean() - Y[T == 0].mean()
    print(f"  朴素 ATE: {naive_ate:.4f}")
    print(f"  偏差: {naive_ate - np.mean(Y1 - Y0):.4f}")
else:
    print("[TODO] 请完成 generate_confounded_data 函数")

## 练习 3.2: DragonNet 架构

现在让我们实现三头龙！

```
输入 X (5维)
    |
    v
[共享表示层]
Linear(5 -> 50) + ELU
Linear(50 -> 25) + ELU
    |
    v
Φ(X) (25维表示)
    |
+---+---+---+
|   |   |   |
v   v   v   v
头0  头1  倾向得分头
|   |       |
v   v       v
Y0  Y1    e(X)∈[0,1]
```

In [ ]:
class SimpleDragonNet(nn.Module):
    """
    简化版 DragonNet - 三头龙网络
    
    与 TARNet 的区别:
    1. 多了一个倾向得分头
    2. 有一个可学习的 epsilon 参数
    3. 使用 ELU 激活函数（原论文推荐）
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 50,
        repr_dim: int = 25
    ):
        super().__init__()
        
        # TODO: 定义共享表示层
        # Input -> Hidden (ELU) -> Representation (ELU)
        # 注意: DragonNet 使用 ELU 激活函数
        self.representation = nn.Sequential(
            # 你的代码
        )
        
        # TODO: 定义控制组输出头 (Y0)
        # Representation -> Hidden (ELU) -> 1
        self.head0 = nn.Sequential(
            # 你的代码
        )
        
        # TODO: 定义处理组输出头 (Y1)
        self.head1 = nn.Sequential(
            # 你的代码
        )
        
        # TODO: 定义倾向得分头 (这是关键！)
        # Representation -> Hidden (ELU) -> 1 (Sigmoid)
        # 注意: 最后使用 Sigmoid 将输出限制在 [0, 1]
        self.propensity_head = nn.Sequential(
            # 你的代码
        )
        
        # TODO: 定义 epsilon 参数 (可学习的标量)
        # 提示: 使用 nn.Parameter(torch.tensor(0.0))
        self.epsilon = None  # 你的代码
    
    def forward(self, x: torch.Tensor) -> Tuple:
        """
        前向传播
        
        Returns:
            (y0_pred, y1_pred, propensity, epsilon, representation)
        """
        # TODO: 完成前向传播
        phi = None      # 共享表示
        y0 = None       # 控制组预测
        y1 = None       # 处理组预测
        propensity = None  # 倾向得分
        
        return y0, y1, propensity, self.epsilon, phi
    
    def predict_ite(self, x: torch.Tensor) -> torch.Tensor:
        """预测个体处理效应"""
        y0, y1, _, _, _ = self.forward(x)
        return y1 - y0

In [ ]:
# 测试 DragonNet 架构
if X is not None:
    model = SimpleDragonNet(input_dim=X.shape[1])
    X_sample = torch.FloatTensor(X[:5])
    
    try:
        y0, y1, prop, eps, phi = model(X_sample)
        if y0 is not None:
            print("DragonNet 架构测试通过! 🐉")
            print(f"\n输出形状:")
            print(f"  Y0 预测: {y0.shape}")
            print(f"  Y1 预测: {y1.shape}")
            print(f"  倾向得分: {prop.shape}")
            print(f"  表示向量: {phi.shape}")
            print(f"\n关键参数:")
            print(f"  Epsilon: {eps}")
            print(f"  倾向得分范围: [{prop.min():.4f}, {prop.max():.4f}]")
        else:
            print("[TODO] 请完成 SimpleDragonNet.forward 函数")
    except Exception as e:
        print(f"[TODO] 请完成 SimpleDragonNet 定义: {e}")

## 练习 3.3: DragonNet 复合损失函数

这是 DragonNet 的精髓！我们需要实现三个损失项：

1. **Factual Loss**: 预测观测结果（和 TARNet 一样）
2. **Propensity Loss**: 预测处理分配
3. **Targeted Regularization**: 魔法调整项

In [ ]:
def dragonnet_loss(
    y_true: torch.Tensor,
    t_true: torch.Tensor,
    y0_pred: torch.Tensor,
    y1_pred: torch.Tensor,
    propensity: torch.Tensor,
    epsilon: torch.Tensor,
    alpha: float = 1.0,
    beta: float = 1.0
) -> Dict[str, torch.Tensor]:
    """
    DragonNet 复合损失函数
    
    L = L_factual + alpha * L_propensity + beta * L_targeted
    
    Parameters:
    -----------
    alpha: 倾向得分损失权重
    beta: targeted regularization 权重
    """
    
    # 确保形状正确
    y0_pred = y0_pred.squeeze()
    y1_pred = y1_pred.squeeze()
    propensity = propensity.squeeze()
    
    # TODO 1: Factual Loss
    # 根据 T 选择对应的预测值，计算 MSE
    # y_pred = T * y1_pred + (1-T) * y0_pred
    y_pred = None  # 你的代码
    factual_loss = None  # 你的代码 (使用 MSE)
    
    # TODO 2: Propensity Loss
    # 二分类交叉熵: BCE(propensity, T)
    # 提示: 使用 F.binary_cross_entropy 或手动计算
    # 添加小常数 1e-8 避免 log(0)
    propensity_loss = None  # 你的代码
    
    # TODO 3: Targeted Regularization (DragonNet 的创新!)
    # h = T / (e(X) + eps) - (1-T) / (1-e(X) + eps)
    # L_targeted = mean((Y - Y_pred - epsilon * h)^2)
    # 
    # 提示:
    # - e(X) 是 propensity
    # - epsilon 是可学习的参数
    # - 添加小常数 1e-8 避免除零
    
    h = None  # 你的代码
    targeted_reg = None  # 你的代码
    
    # TODO: 总损失
    total_loss = None  # 你的代码
    
    return {
        'total': total_loss,
        'factual': factual_loss,
        'propensity': propensity_loss,
        'targeted': targeted_reg
    }

In [ ]:
# 测试损失函数
try:
    y_true = torch.FloatTensor([1.0, 2.0, 3.0, 4.0, 5.0])
    t_true = torch.FloatTensor([1.0, 0.0, 1.0, 0.0, 1.0])
    y0_pred = torch.FloatTensor([1.5, 2.0, 2.5, 4.5, 4.0])
    y1_pred = torch.FloatTensor([2.0, 2.5, 3.0, 5.0, 5.5])
    propensity = torch.FloatTensor([0.7, 0.3, 0.6, 0.4, 0.8])
    epsilon = torch.tensor(0.1)
    
    losses = dragonnet_loss(
        y_true, t_true, y0_pred, y1_pred,
        propensity, epsilon, alpha=1.0, beta=1.0
    )
    
    if losses['total'] is not None:
        print("损失函数测试通过! 🎉")
        print(f"\n各损失项:")
        print(f"  Factual Loss: {losses['factual'].item():.4f}")
        print(f"  Propensity Loss: {losses['propensity'].item():.4f}")
        print(f"  Targeted Reg: {losses['targeted'].item():.4f}")
        print(f"  Total Loss: {losses['total'].item():.4f}")
    else:
        print("[TODO] 请完成 dragonnet_loss 函数")
except Exception as e:
    print(f"[TODO] 请完成 dragonnet_loss 函数: {e}")

## 练习 3.4: 训练 DragonNet

让我们把三头龙训练起来！

In [ ]:
def train_dragonnet(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_epochs: int = 200,
    batch_size: int = 64,
    learning_rate: float = 1e-3,
    alpha: float = 1.0,
    beta: float = 1.0,
    verbose: bool = True
) -> Tuple[SimpleDragonNet, dict]:
    """
    训练 DragonNet
    
    Parameters:
    -----------
    alpha: 倾向得分损失权重
    beta: targeted regularization 权重
    
    Returns:
        (trained_model, training_history)
    """
    
    # 数据准备
    X_tensor = torch.FloatTensor(X)
    T_tensor = torch.FloatTensor(T)
    Y_tensor = torch.FloatTensor(Y)
    
    dataset = TensorDataset(X_tensor, T_tensor, Y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # 初始化模型
    model = SimpleDragonNet(input_dim=X.shape[1])
    
    # TODO: 定义优化器 (Adam, 学习率=learning_rate)
    optimizer = None  # 你的代码
    
    # 训练历史
    history = {
        'total_loss': [],
        'factual_loss': [],
        'propensity_loss': [],
        'targeted_loss': [],
        'epsilon': []
    }
    
    # 训练循环
    for epoch in range(n_epochs):
        epoch_losses = {k: 0 for k in history.keys() if k != 'epsilon'}
        n_batches = 0
        
        for batch_x, batch_t, batch_y in dataloader:
            # TODO: 完成训练步骤
            # 1. 清零梯度
            # 2. 前向传播
            # 3. 计算 DragonNet 损失
            # 4. 反向传播
            # 5. 更新参数
            # 6. 记录各项损失
            
            # 你的代码
            pass
        
        # 记录平均损失
        for k in epoch_losses.keys():
            history[k.replace('_loss', '_loss')].append(
                epoch_losses[k] / n_batches if n_batches > 0 else 0
            )
        
        # 记录 epsilon
        if model.epsilon is not None:
            history['epsilon'].append(model.epsilon.item())
        
        if verbose and (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}")
            print(f"  Total: {history['total_loss'][-1]:.4f}")
            print(f"  Factual: {history['factual_loss'][-1]:.4f}")
            print(f"  Propensity: {history['propensity_loss'][-1]:.4f}")
            if model.epsilon is not None:
                print(f"  Epsilon: {model.epsilon.item():.4f}")
    
    return model, history

In [ ]:
# 训练 DragonNet
if X is not None:
    print("开始训练 DragonNet...")
    print("="*50)
    
    try:
        model, history = train_dragonnet(
            X, T, Y,
            n_epochs=200,
            batch_size=64,
            alpha=1.0,
            beta=1.0
        )
        
        if model is not None and len(history['total_loss']) > 0:
            print("\n训练完成! 🐉")
            print(f"最终总损失: {history['total_loss'][-1]:.4f}")
        else:
            print("[TODO] 请完成 train_dragonnet 函数")
    except Exception as e:
        print(f"[TODO] 训练出错: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
# 可视化训练过程
if 'history' in dir() and history is not None and len(history['total_loss']) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 总损失
    axes[0, 0].plot(history['total_loss'], 'b-', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Total Loss')
    axes[0, 0].set_title('Total Loss 收敛曲线')
    
    # 三个损失项
    axes[0, 1].plot(history['factual_loss'], 'g-', label='Factual', linewidth=2)
    axes[0, 1].plot(history['propensity_loss'], 'r-', label='Propensity', linewidth=2)
    if 'targeted_loss' in history and len(history['targeted_loss']) > 0:
        axes[0, 1].plot(history['targeted_loss'], 'purple', label='Targeted', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title('各损失项收敛曲线')
    axes[0, 1].legend()
    
    # Epsilon 变化
    if len(history['epsilon']) > 0:
        axes[1, 0].plot(history['epsilon'], 'orange', linewidth=2)
        axes[1, 0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Epsilon')
        axes[1, 0].set_title('Epsilon 参数变化 (可学习的调整项)')
    
    # 损失比例
    if len(history['factual_loss']) > 0:
        total = np.array(history['total_loss'])
        factual = np.array(history['factual_loss'])
        propensity = np.array(history['propensity_loss'])
        
        axes[1, 1].stackplot(
            range(len(total)),
            factual, propensity,
            labels=['Factual', 'Propensity'],
            colors=['#2ecc71', '#e74c3c'],
            alpha=0.7
        )
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].set_title('损失组成堆叠图')
        axes[1, 1].legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

## 练习 3.5: 评估倾向得分估计质量

DragonNet 的倾向得分头学得好不好？让我们来评估一下！

In [ ]:
def evaluate_propensity_score(
    model: SimpleDragonNet,
    X: np.ndarray,
    T: np.ndarray
) -> dict:
    """
    评估倾向得分估计质量
    
    评估指标:
    1. AUC: 预测处理分配的能力
    2. 校准曲线: 预测概率 vs 实际处理率
    """
    from sklearn.metrics import roc_auc_score
    
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        _, _, propensity_pred, _, _ = model(X_tensor)
        propensity_pred = propensity_pred.numpy().squeeze()
    
    # TODO: 计算 AUC
    # 提示: 使用 roc_auc_score(T, propensity_pred)
    auc = None  # 你的代码
    
    # TODO: 计算校准指标
    # 将样本按预测倾向得分分成 5 组
    # 对比每组的平均预测倾向得分和实际处理率
    n_bins = 5
    calibration = []
    
    for i in range(n_bins):
        # 你的代码
        # 找出倾向得分在第 i 个分位区间的样本
        # 计算该区间的平均预测倾向得分和实际处理率
        pass
    
    return {
        'auc': auc,
        'calibration': calibration,
        'propensity_pred': propensity_pred,
        'propensity_mean': propensity_pred.mean(),
        'propensity_std': propensity_pred.std()
    }

In [ ]:
# 评估倾向得分
if 'model' in dir() and model is not None:
    prop_metrics = evaluate_propensity_score(model, X, T)
    
    if prop_metrics['auc'] is not None:
        print("倾向得分评估结果:")
        print(f"  AUC: {prop_metrics['auc']:.4f}")
        print(f"  均值: {prop_metrics['propensity_mean']:.4f}")
        print(f"  标准差: {prop_metrics['propensity_std']:.4f}")
        print(f"  实际处理率: {T.mean():.4f}")
        
        # 可视化倾向得分分布
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # 分组分布
        axes[0].hist(
            prop_metrics['propensity_pred'][T == 0],
            bins=30, alpha=0.6, label='控制组', color='#3498db'
        )
        axes[0].hist(
            prop_metrics['propensity_pred'][T == 1],
            bins=30, alpha=0.6, label='处理组', color='#e74c3c'
        )
        axes[0].set_xlabel('预测倾向得分')
        axes[0].set_ylabel('频数')
        axes[0].set_title('倾向得分分布 (按处理分组)')
        axes[0].legend()
        
        # 校准曲线
        if len(prop_metrics['calibration']) > 0:
            pred_props = [c['pred_mean'] for c in prop_metrics['calibration']]
            actual_rates = [c['actual_rate'] for c in prop_metrics['calibration']]
            
            axes[1].scatter(pred_props, actual_rates, s=100, c='#2ecc71', edgecolors='black')
            axes[1].plot([0, 1], [0, 1], 'k--', label='完美校准')
            axes[1].set_xlabel('预测倾向得分均值')
            axes[1].set_ylabel('实际处理率')
            axes[1].set_title('倾向得分校准曲线')
            axes[1].legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("[TODO] 请完成 evaluate_propensity_score 函数")

## 练习 3.6: 评估因果效应估计

现在让我们看看 DragonNet 的因果效应估计有多准！

In [ ]:
def evaluate_causal_effect(
    model: SimpleDragonNet,
    X: np.ndarray,
    Y0: np.ndarray,
    Y1: np.ndarray
) -> dict:
    """
    评估因果效应估计质量
    """
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        y0_pred, y1_pred, _, _, _ = model(X_tensor)
        y0_pred = y0_pred.numpy().squeeze()
        y1_pred = y1_pred.numpy().squeeze()
    
    # 真实 ITE
    ite_true = Y1 - Y0
    ite_pred = y1_pred - y0_pred
    
    # PEHE (Precision in Estimation of Heterogeneous Effect)
    pehe = np.sqrt(np.mean((ite_true - ite_pred) ** 2))
    
    # ATE 误差
    ate_true = np.mean(ite_true)
    ate_pred = np.mean(ite_pred)
    ate_error = np.abs(ate_true - ate_pred)
    
    # ITE 相关性
    ite_corr = np.corrcoef(ite_true.flatten(), ite_pred.flatten())[0, 1]
    
    return {
        'pehe': pehe,
        'ate_true': ate_true,
        'ate_pred': ate_pred,
        'ate_error': ate_error,
        'ite_corr': ite_corr,
        'ite_true': ite_true,
        'ite_pred': ite_pred
    }

In [ ]:
# 评估因果效应
if 'model' in dir() and model is not None and Y0 is not None:
    causal_metrics = evaluate_causal_effect(model, X, Y0, Y1)
    
    print("因果效应估计结果:")
    print(f"  PEHE: {causal_metrics['pehe']:.4f}")
    print(f"  真实 ATE: {causal_metrics['ate_true']:.4f}")
    print(f"  预测 ATE: {causal_metrics['ate_pred']:.4f}")
    print(f"  ATE 误差: {causal_metrics['ate_error']:.4f}")
    print(f"  ITE 相关性: {causal_metrics['ite_corr']:.4f}")
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # ITE 散点图
    axes[0].scatter(
        causal_metrics['ite_true'],
        causal_metrics['ite_pred'],
        alpha=0.3, c='#3498db'
    )
    min_val = min(causal_metrics['ite_true'].min(), causal_metrics['ite_pred'].min())
    max_val = max(causal_metrics['ite_true'].max(), causal_metrics['ite_pred'].max())
    axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', label='完美预测')
    axes[0].set_xlabel('真实 ITE')
    axes[0].set_ylabel('预测 ITE')
    axes[0].set_title(f'ITE 预测 vs 真实 (相关性: {causal_metrics["ite_corr"]:.3f})')
    axes[0].legend()
    
    # ITE 误差分布
    ite_error = causal_metrics['ite_pred'] - causal_metrics['ite_true']
    axes[1].hist(ite_error, bins=50, alpha=0.7, color='#9b59b6', edgecolor='black')
    axes[1].axvline(x=0, color='red', linestyle='--', label='零误差')
    axes[1].axvline(x=ite_error.mean(), color='orange', linestyle='-', label=f'均值: {ite_error.mean():.3f}')
    axes[1].set_xlabel('ITE 预测误差')
    axes[1].set_ylabel('频数')
    axes[1].set_title('ITE 预测误差分布')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 练习 3.7: TARNet vs DragonNet 对比实验

DragonNet 比 TARNet 好在哪里？让我们来做个对比实验！

In [ ]:
def compare_tarnet_dragonnet(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    Y0: np.ndarray,
    Y1: np.ndarray,
    n_epochs: int = 200
) -> dict:
    """
    对比 TARNet 和 DragonNet 的性能
    
    TARNet = DragonNet with alpha=0, beta=0 (只用 Factual Loss)
    DragonNet = DragonNet with alpha=1, beta=1 (完整损失)
    """
    
    # TODO: 训练 TARNet (alpha=0, beta=0)
    # 提示: 使用 train_dragonnet 但设置 alpha=0, beta=0
    print("训练 TARNet (只用 Factual Loss)...")
    tarnet, _ = None, None  # 你的代码
    
    # TODO: 训练 DragonNet (alpha=1.0, beta=1.0)
    print("\n训练 DragonNet (完整损失)...")
    dragonnet, _ = None, None  # 你的代码
    
    # 评估函数
    def evaluate_model(model):
        if model is None:
            return None
        model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X)
            y0_pred, y1_pred, _, _, _ = model(X_tensor)
            y0_pred = y0_pred.numpy().squeeze()
            y1_pred = y1_pred.numpy().squeeze()
        
        ite_true = Y1 - Y0
        ite_pred = y1_pred - y0_pred
        pehe = np.sqrt(np.mean((ite_true - ite_pred) ** 2))
        
        ate_true = np.mean(Y1 - Y0)
        ate_pred = np.mean(y1_pred - y0_pred)
        ate_error = np.abs(ate_true - ate_pred)
        
        return {'pehe': pehe, 'ate_error': ate_error}
    
    tarnet_metrics = evaluate_model(tarnet)
    dragonnet_metrics = evaluate_model(dragonnet)
    
    return {
        'tarnet': tarnet_metrics,
        'dragonnet': dragonnet_metrics
    }

In [ ]:
# 对比实验
if X is not None and Y0 is not None:
    print("="*60)
    print("TARNet vs DragonNet 对比实验")
    print("="*60)
    
    try:
        comparison = compare_tarnet_dragonnet(X, T, Y, Y0, Y1, n_epochs=200)
        
        if comparison['tarnet'] is not None and comparison['dragonnet'] is not None:
            print("\n" + "="*60)
            print("对比结果:")
            print("="*60)
            print(f"\n{'模型':<15} {'PEHE':<15} {'ATE 误差':<15}")
            print("-"*45)
            print(f"{'TARNet':<15} {comparison['tarnet']['pehe']:<15.4f} {comparison['tarnet']['ate_error']:<15.4f}")
            print(f"{'DragonNet':<15} {comparison['dragonnet']['pehe']:<15.4f} {comparison['dragonnet']['ate_error']:<15.4f}")
            
            # 计算改进
            pehe_improve = (comparison['tarnet']['pehe'] - comparison['dragonnet']['pehe']) / comparison['tarnet']['pehe'] * 100
            ate_improve = (comparison['tarnet']['ate_error'] - comparison['dragonnet']['ate_error']) / comparison['tarnet']['ate_error'] * 100
            
            print(f"\n改进:")
            print(f"  PEHE: {pehe_improve:+.1f}%")
            print(f"  ATE 误差: {ate_improve:+.1f}%")
            
            # 可视化对比
            fig, ax = plt.subplots(figsize=(10, 6))
            
            x = np.arange(2)
            width = 0.35
            
            tarnet_vals = [comparison['tarnet']['pehe'], comparison['tarnet']['ate_error']]
            dragonnet_vals = [comparison['dragonnet']['pehe'], comparison['dragonnet']['ate_error']]
            
            bars1 = ax.bar(x - width/2, tarnet_vals, width, label='TARNet', color='#3498db')
            bars2 = ax.bar(x + width/2, dragonnet_vals, width, label='DragonNet', color='#e74c3c')
            
            ax.set_ylabel('误差值')
            ax.set_title('TARNet vs DragonNet 性能对比')
            ax.set_xticks(x)
            ax.set_xticklabels(['PEHE', 'ATE Error'])
            ax.legend()
            
            # 添加数值标签
            for bar, val in zip(bars1, tarnet_vals):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                       f'{val:.3f}', ha='center', va='bottom')
            for bar, val in zip(bars2, dragonnet_vals):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                       f'{val:.3f}', ha='center', va='bottom')
            
            plt.tight_layout()
            plt.show()
        else:
            print("[TODO] 请完成 compare_tarnet_dragonnet 函数")
    except Exception as e:
        print(f"[TODO] 对比实验出错: {e}")
        import traceback
        traceback.print_exc()

## 🤔 思考题

在代码单元格中写下你的答案：

In [ ]:
# 思考题 1: 倾向得分头在 DragonNet 中的作用是什么？
# 为什么不单独训练一个倾向得分模型？

answer_1 = """
你的答案:



"""
print(answer_1)

In [ ]:
# 思考题 2: 为什么 Targeted Regularization 中需要 h = T/e(X) - (1-T)/(1-e(X))?
# 这个公式的直觉是什么？

answer_2 = """
你的答案:



"""
print(answer_2)

In [ ]:
# 思考题 3: epsilon 参数的作用是什么？为什么它是可学习的？

answer_3 = """
你的答案:



"""
print(answer_3)

In [ ]:
# 思考题 4: 什么时候 DragonNet 会比 TARNet 效果更好？

answer_4 = """
你的答案:



"""
print(answer_4)

In [ ]:
# 思考题 5: 如果倾向得分头的预测很差，会对整个模型有什么影响？

answer_5 = """
你的答案:



"""
print(answer_5)

In [ ]:
# 思考题 6: DragonNet 和传统的倾向得分方法（如 IPW）有什么区别？

answer_6 = """
你的答案:



"""
print(answer_6)

In [ ]:
# 思考题 7: 在实践中，如何选择 alpha 和 beta 这两个超参数？

answer_7 = """
你的答案:



"""
print(answer_7)

## 📊 总结

### DragonNet 核心概念

| 组件 | 作用 | 关键点 |
|------|------|--------|
| 共享表示层 | 学习平衡表示 | 同时用于预测结果和倾向得分 |
| Y(0)头 | 预测控制组结果 | 与 TARNet 相同 |
| Y(1)头 | 预测处理组结果 | 与 TARNet 相同 |
| 倾向得分头 | 预测处理概率 | 帮助学习混淆因子 |
| epsilon | 调整参数 | 可学习，自动适应偏差 |

### 损失函数对比

| 模型 | 损失函数 |
|------|----------|
| TARNet | $\mathcal{L}_{\text{factual}}$ |
| DragonNet | $\mathcal{L}_{\text{factual}} + \alpha \mathcal{L}_{\text{propensity}} + \beta \mathcal{L}_{\text{targeted}}$ |

### 什么时候用 DragonNet?

✅ **适合场景**:
- 存在强混淆
- 处理分配高度不平衡
- 需要更稳健的估计

❌ **不适合场景**:
- 随机对照实验（无混淆）
- 数据量非常小（复杂模型容易过拟合）

## 🚀 进阶挑战

如果你已经完成了基础练习，可以尝试以下挑战：

1. **实现 CEVAE** (Causal Effect Variational Autoencoder)
2. **添加正则化**: 尝试添加 L2 正则化或 Dropout
3. **多处理组扩展**: 将 DragonNet 扩展到多个处理组
4. **不确定性估计**: 添加 Bayesian 方法估计 CATE 的不确定性

In [ ]:
print("🐉 恭喜完成 DragonNet 练习!")
print("\n你已经学会了:")
print("  ✓ DragonNet 的三头架构")
print("  ✓ 倾向得分头的作用")
print("  ✓ Targeted Regularization 的原理")
print("  ✓ TARNet vs DragonNet 的对比")
print("\n下一步: Chapter 5 异质效应分析 - 因果森林!")